# Homework 4

Before you run your homework run the code below:

In [146]:
import os
import sys
import hashlib
import pandas as pd
import numpy as np
import glob
import re
from urllib.request import urlopen
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn import datasets, linear_model


assert os.path.exists('../homeworks/HW4.ipynb')

info = sys.platform + '\n' + sys.version + '\n' + os.getcwd() + '\n' + os.getlogin()
name = hashlib.sha256(info.encode('utf-8')).hexdigest()
with open('../other/hw-4-'+name,'w') as f:
    f.write(info)

In [147]:
with open('../other/hw-4-'+name,'r') as f:
    info = f.read()
name = hashlib.sha256(info.encode('utf-8')).hexdigest()
assert os.path.exists('../other/hw-4-'+name)

## Task

Apply all of the supervised and unsupervised classification and clustering algorithms we learned so far for the [sonar dataset from UCI](http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)).

The code for downloading the data is below. Don't load the data again and again in each subtask, refer the data as `SONAR` after you run the cell below.

In [148]:
with urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data") as fil:
    SONAR = pd.read_csv(fil, delimiter=',', header=None)

# SONAR.head()
# import matplotlib.pyplot as plot
# SONAR.plot()
# plot.show()

# datayi %25 test datasi olacak sekilde boluyoruz
Xtrain, Xtest, Ytrain, Ytest = train_test_split(SONAR.iloc[:,0:60], SONAR.iloc[:,60], test_size=0.25)

Split the tasks into subtasks and into separate cells.  Also before your analysis for each subtask, write a short paragraph before explaining which algorithm you are going to use. Each subtask should look like this

## Subtask 1: K-Means

In [149]:

model = KMeans(n_clusters=2, random_state=0)
# model should be fit with train data
model.fit(Xtrain)
# predict with test data
predicted = model.predict(Xtest)
labels = {"R":1, "M":0}
# accuracy sonucu asagidaki durumda daha kotu bunun sebebi label'lari yanlis tahmin etmis olmamiz
# labels = {"R":0, "M":1}
real = Ytest.map(lambda x: labels[x])
cm = confusion_matrix(real,predicted)
print(cm)
accuracy = accuracy_score(real,predicted)

print(accuracy)

[[14 19]
 [ 8 11]]
0.4807692307692308


In [150]:
## YOUR CODE

## Subtask 2: KNN

In [151]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(Xtrain,Ytrain)
predicted = model.predict(Xtest)
cm = confusion_matrix(Ytest,predicted)
print(cm)
accuracy = accuracy_score(Ytest,predicted)
print(accuracy)

[[29  4]
 [ 4 15]]
0.8461538461538461


In [152]:
## YOUR CODE

## Subtask 3: Naive Bayes Classifier


In [153]:
from sklearn.naive_bayes import GaussianNB

# create naive bayes classifier
gaunb = GaussianNB()

# train classifier with dataset
gaunb = gaunb.fit(Xtrain, Ytrain)

# predict using classifier
predicted = gaunb.predict(Xtest)
cm = confusion_matrix(Ytest,predicted)
print(cm)
accuracy = accuracy_score(Ytest,predicted)
print(accuracy)

[[17 16]
 [ 3 16]]
0.6346153846153846


## Subtask 4: Linear Regression



In [156]:
model = linear_model.LinearRegression()

# Train the model using the training sets
# Ytrain2 = Ytrain.replace(to_replace=['M', 'R'], value=[0, 1])
# Ytest2 = Ytest.replace(to_replace=['M', 'R'], value=[0, 1])
model.fit(Xtrain, Ytrain)

# Make predictions using the testing set
predicted = model.predict(Ytest)
print(predicted)
cm = confusion_matrix(Ytest,predicted)
print(cm)
accuracy = accuracy_score(Ytest2,predicted)
print(accuracy)

ValueError: Expected 2D array, got 1D array instead:
array=[0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0
 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.